In [ ]:
# ===============================================================================================================#
# Copyright 2022 Infosys Ltd.                                                                          #
# Use of this source code is governed by Apache License Version 2.0 that can be found in the LICENSE file or at  #
# http://www.apache.org/licenses/                                                                                #
# ===============================================================================================================#

## 1. Preparation

In [ ]:
import json
import time
from _internal_utils.demo_helper import DemoHelper
import pandas as pd
from IPython.display import Image

#### <span style='color:blue'>Enter image paths</span>

In [ ]:
IMAGE_PATHS=['./data/coi_1.jpg','./data/coi_2.jpg']

#### Visualize

In [ ]:
tab_widget = DemoHelper.create_tab_toolbar(IMAGE_PATHS)
display(tab_widget)
for idx, image_path in enumerate(IMAGE_PATHS):
    with tab_widget.children[idx]:
        display(Image(filename=image_path, width=1000, height=50))   

## 2. Initialization

#### <span style='color:blue'>Create new instance with desired OCR service provider</span>

In [ ]:
import json
import logging
import os

from infy_ocr_generator import ocr_generator
from infy_ocr_generator.providers.azure_read_ocr_data_service_provider import AzureReadOcrDataServiceProvider

if not os.path.exists("./logs"):
    os.makedirs("./logs")
logging.basicConfig(
    filename=("./logs" + "/app_log.log"),
    format="%(asctime)s- %(levelname)s- %(message)s",
    level=logging.INFO,
    datefmt="%d-%b-%y %H:%M:%S",
)
logger = logging.getLogger()

CONFIG_PARAMS_DICT = {
    'azure': {
        'computer_vision': {
            'subscription_key': '<Enter API key>',
            'api_read': {
                'url': 'https://<Enter host name>/vision/v3.2/read/analyze'
            }
        }
    }
}

azure_read_provider = AzureReadOcrDataServiceProvider(
    config_params_dict=CONFIG_PARAMS_DICT,
    overwrite=True,
    logger=logger
)
ocr_generator_obj = ocr_generator.OcrGenerator(data_service_provider=azure_read_provider)

## 3. API - submit_request(), receive_response() AND generate()
**_Note: These APIs must be called in sequence and as a group_**

### 3.1. Example

In [ ]:
submit_req_result = ocr_generator_obj.submit_request(
    doc_data_list=[
        {
            "doc_path":doc_file_path,
            "pages": index+1
        } for index, doc_file_path in enumerate(IMAGE_PATHS)
    ]
)

##### <span style='color:blue'>Wait for few seconds before next call...</span>

In [ ]:
time.sleep(7)

In [ ]:
re_res_result = ocr_generator_obj.receive_response(submit_req_result)

In [ ]:
ocr_result_list = ocr_generator_obj.generate(
    api_response_list=re_res_result
)

#### Output

In [ ]:
print(json.dumps(ocr_result_list, indent=4))

#### Visualize

In [ ]:
OCR_PATHS = [x['output_doc'] for x in ocr_result_list]
tab_widget = DemoHelper.create_tab_toolbar(OCR_PATHS)
display(tab_widget)
for idx, ocr_path in enumerate(OCR_PATHS):
    with tab_widget.children[idx]:
        print(DemoHelper.get_shortened_text(DemoHelper.read_file(ocr_path),max_line_count=50))